In [1]:
!pip install yfinance

In [2]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [4]:
symbol = input("Enter stock symbol (e.g., RELIANCE, TCS, INFY): ").upper()
exchange = input("Enter exchange (NS for NSE, BO for BSE): ").upper()
ticker = f"{symbol}.{exchange}"

start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = input("Enter end date (YYYY-MM-DD): ")
data = yf.download(ticker, start=start_date, end=end_date)

data.head()

Enter stock symbol (e.g., RELIANCE, TCS, INFY):  reliance
Enter exchange (NS for NSE, BO for BSE):  ns
Enter start date (YYYY-MM-DD):  2024-06-01
Enter end date (YYYY-MM-DD):  2025-06-30


C:\Users\abali\AppData\Local\Temp\ipykernel_10408\2297851698.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2024-06-03,1499.228516,1503.372890,1448.280652,1472.104322,21527942
2024-06-04,1387.009155,1487.043779,1349.313184,1487.043779,36709098
2024-06-05,1410.311646,1420.635260,1373.930892,1411.552462,17464890
2024-06-06,1421.081909,1433.092985,1410.907217,1424.456953,17855722
2024-06-07,1459.150024,1461.185011,1416.019306,1418.004612,18558696


In [9]:
stock = yf.Ticker(ticker)

# Fetch fundamentals
eps = stock.info.get("trailingEps", None)            # Earnings per Share
pe_ratio = stock.info.get("trailingPE", None)        # Price to Earnings ratio
roe = stock.info.get("returnOnEquity", None)         # Return on Equity (fraction, e.g., 0.18 = 18%)
de_ratio = stock.info.get("debtToEquity", None)      # Debt-to-Equity ratio

print("\nFundamental Metrics:")
print(f"EPS: {eps}")
print(f"P/E Ratio: {pe_ratio}")
print(f"ROE: {roe}")
print(f"Debt-to-Equity: {de_ratio}")


Fundamental Metrics:
EPS: 60.24
P/E Ratio: 22.971447
ROE: None
Debt-to-Equity: 36.605


In [10]:
# --- Moving Averages ---
data["MA50"] = data["Close"].rolling(50).mean()
data["MA200"] = data["Close"].rolling(200).mean()

# --- RSI Calculation ---
delta = data["Close"].diff()
gain = (delta.where(delta > 0, 0)).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()

rs = gain / loss
data["RSI"] = 100 - (100 / (1 + rs))

print("\nLatest Technical Indicators:")
print(f"50-day MA: {data['MA50'].iloc[-1]}")
print(f"200-day MA: {data['MA200'].iloc[-1]}")
print(f"RSI: {data['RSI'].iloc[-1]}")


Latest Technical Indicators:
50-day MA: nan
200-day MA: nan
RSI: nan


In [11]:
def evaluate_stock(eps, pe_ratio, roe, de_ratio, ma50, ma200, rsi):
    score = 0

    # Fundamentals
    if eps and eps > 0: score += 1
    if pe_ratio and pe_ratio < 20: score += 1
    if roe and roe > 0.15: score += 1
    if de_ratio and de_ratio < 1: score += 1

    # Technicals
    if ma50 > ma200: score += 1
    if rsi and rsi < 70 and rsi > 30: score += 1

    # Final decision
    if score >= 5:
        return " Strong Buy"
    elif score >= 3:
        return " Buy / Hold"
    else:
        return " Avoid / Weak"

In [12]:
result = evaluate_stock(eps, pe_ratio, roe, de_ratio, data["MA50"].iloc[-1], data["MA200"].iloc[-1], data["RSI"].iloc[-1])
print("Final Recommendation:", result)

Final Recommendation:  Avoid / Weak
